Тема «POS-tagger и NER»

Задание 1. Написать теггер на данных с русским языком
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы, сделать выводы  


Задание 2. Проверить, насколько хорошо работает NER
Данные брать из Index of /pub/named_entities
1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы.
  - передаём в сетку токен и его соседей.
  - передаём в сетку только токен.
  - свой вариант.
3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

In [ ]:
%pip install pyconll
%pip install corus
%pip install -U spacy
%pip install slovnet
%pip install deeppavlov
%pip install ipymarkup
%pip install sklearn_crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 361.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.6.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.3/468.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
!mkdir datasets
!wget https://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip -d ./datasets

--2023-08-04 12:29:21--  https://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.26MB/s    in 1.4s    

2023-08-04 12:29:24 (1.26 MB/s) - ‘collection5.zip’ saved [1899530/1899530]

Archive:  collection5.zip
   creating: ./datasets/Collection5/
  inflating: ./datasets/Collection5/001.ann  
  inflating: ./datasets/Collection5/001.txt  
  inflating: ./datasets/Collection5/002.ann  
  inflating: ./datasets/Collection5/002.txt  
  inflating: ./datasets/Collection5/003.ann  
  inflating: ./datasets/Collection5/003.txt  
  inflating: ./datasets/Collection5/004.ann  
  inflating: ./datasets/Collection5/004.txt  
  inflating: ./datasets/Collection5/005.ann  
  inflating: .

In [ ]:
# SynTagRus is now split into 3 parts. Download them all
!wget -O ./datasets/ru_syntagrus-ud-train-a.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./datasets/ru_syntagrus-ud-train-b.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget -O ./datasets/ru_syntagrus-ud-train-c.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
# Concatenate files together to get the full training set
!cat ./datasets/ru_syntagrus-ud-train-a.conllu ./datasets/ru_syntagrus-ud-train-b.conllu ./datasets/ru_syntagrus-ud-train-c.conllu > ./datasets/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2023-08-04 12:29:25--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train-a.conllu’

./datasets/ru_synta 100%[===================>]  38.85M  --.-KB/s    in 0.1s    

2023-08-04 12:29:28 (354 MB/s) - ‘./datasets/ru_syntagrus-ud-train-a.conllu’ saved [40736581/40736581]

--2023-08-04 12:29:28--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.gith

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import pyconll
import nltk
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
import os
import corus
import deeppavlov
from deeppavlov import configs, build_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
import spacy
from spacy import displacy
import pandas as pd
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import tokenize
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

### Задание 1.

In [ ]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [ ]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])

fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])

fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [ ]:
all_train_texts = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_test]

all_train_labels = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_test]

In [ ]:
test_sent = all_test_texts[20]
test_data = fdata_test

In [ ]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(test_sent.lower().split()), unigram_tagger.evaluate(test_data))

[('таким', 'DET'),
 ('образом', 'NOUN'),
 (',', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('выдаёт', None),
 ('один', 'NUM'),
 ('и', 'CCONJ'),
 ('тот', 'DET'),
 ('же', 'PART'),
 ('результат', 'NOUN'),
 ('(', 'PUNCT'),
 ('ответ', 'NOUN'),
 (')', 'PUNCT'),
 ('для', 'ADP'),
 ('одних', 'DET'),
 ('и', 'CCONJ'),
 ('тех', 'DET'),
 ('же', 'PART'),
 ('исходных', 'ADJ'),
 ('данных', 'NOUN'),
 ('.', 'PUNCT')]

0.8782863467673677

In [ ]:
bigram_tagger = BigramTagger(fdata_train)
display(bigram_tagger.tag(test_sent.lower().split()), bigram_tagger.evaluate(test_data))

[('таким', None),
 ('образом', None),
 (',', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('выдаёт', None),
 ('один', 'NUM'),
 ('и', 'CCONJ'),
 ('тот', 'DET'),
 ('же', 'PART'),
 ('результат', 'NOUN'),
 ('(', 'PUNCT'),
 ('ответ', 'NOUN'),
 (')', 'PUNCT'),
 ('для', 'ADP'),
 ('одних', 'DET'),
 ('и', 'CCONJ'),
 ('тех', 'DET'),
 ('же', 'PART'),
 ('исходных', None),
 ('данных', None),
 ('.', 'PUNCT')]

0.7101308678950452

In [ ]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent.lower().split()), bigram_tagger.evaluate(test_data))

[('таким', 'DET'),
 ('образом', 'NOUN'),
 (',', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('выдаёт', None),
 ('один', 'NUM'),
 ('и', 'CCONJ'),
 ('тот', 'DET'),
 ('же', 'PART'),
 ('результат', 'NOUN'),
 ('(', 'PUNCT'),
 ('ответ', 'NOUN'),
 (')', 'PUNCT'),
 ('для', 'ADP'),
 ('одних', 'DET'),
 ('и', 'CCONJ'),
 ('тех', 'DET'),
 ('же', 'PART'),
 ('исходных', 'ADJ'),
 ('данных', 'NOUN'),
 ('.', 'PUNCT')]

0.8839768214076438

In [ ]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

[('таким', None),
 ('образом', None),
 (',', 'PUNCT'),
 ('алгоритм', None),
 ('выдаёт', None),
 ('один', None),
 ('и', 'PART'),
 ('тот', None),
 ('же', None),
 ('результат', None),
 ('(', None),
 ('ответ', None),
 (')', None),
 ('для', 'ADP'),
 ('одних', None),
 ('и', None),
 ('тех', None),
 ('же', None),
 ('исходных', None),
 ('данных', None),
 ('.', 'PUNCT')]

0.4067191874470994

In [ ]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

[('таким', 'DET'),
 ('образом', 'NOUN'),
 (',', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('выдаёт', None),
 ('один', 'NUM'),
 ('и', 'CCONJ'),
 ('тот', 'DET'),
 ('же', 'PART'),
 ('результат', 'NOUN'),
 ('(', 'PUNCT'),
 ('ответ', 'NOUN'),
 (')', 'PUNCT'),
 ('для', 'ADP'),
 ('одних', 'DET'),
 ('и', 'CCONJ'),
 ('тех', 'DET'),
 ('же', 'PART'),
 ('исходных', 'ADJ'),
 ('данных', 'NOUN'),
 ('.', 'PUNCT')]

0.8830522820496126

In [ ]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN')
tag = backoff_tagger(fdata_train,
                     [UnigramTagger, BigramTagger, TrigramTagger],
                     backoff = backoff)


In [ ]:
tag.evaluate(test_data)

0.8825379256462009

Комбинация тэггеров дает результат лучше, чем UnigramTagger, BigramTagger, TrigramTagger по отдельности. \
Лучший результат показал BigramTagger, у которого в качестве backoff был UnigramTagger.

In [ ]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        if (tok is None or tok[0] is None):
          continue
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])

test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        if (tok is None or tok[0] is None):
          continue
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [ ]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [ ]:
vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'),
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'),
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)]
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'),
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'),
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)]

In [ ]:
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)

    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels, pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels, pred)
    accuracy_scores.append(acc)

    print(vectorizer)
    print(classification_report(test_enc_labels, pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.91      0.91      0.91     15103
         ADP       0.99      1.00      0.99     13717
         ADV       0.90      0.89      0.90      7783
         AUX       0.81      0.97      0.88      1390
       CCONJ       0.88      0.98      0.93      5672
         DET       0.83      0.78      0.80      4265
        INTJ       0.28      0.29      0.29        24
        NOUN       0.92      0.95      0.94     36238
      NO_TAG       1.00      1.00      1.00       204
         NUM       0.87      0.89      0.88      1734
        PART       0.94      0.77      0.85      5125
        PRON       0.86      0.88      0.87      7444
       PROPN       0.78      0.58      0.67      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.81      0.90      0.85      2865
         SYM       1.00      0.85      0.92        62
        VERB       0.94     

In [ ]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927277
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.921265
2,"HashingVectorizer(analyzer='char', n_features=...",0.877628
3,"CountVectorizer(ngram_range=(1, 3))",0.661288
4,"TfidfVectorizer(ngram_range=(1, 3))",0.659126
5,"HashingVectorizer(n_features=1000, ngram_range...",0.343001


In [ ]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.928880
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.922855
2,"HashingVectorizer(analyzer='char', n_features=...",0.880948
3,"CountVectorizer(ngram_range=(1, 3))",0.652483
4,"TfidfVectorizer(ngram_range=(1, 3))",0.649740
5,"HashingVectorizer(n_features=1000, ngram_range...",0.363710


Логистическая регрессия работает заметно медленнее, но показывает большую точность.

### Задание 2.

In [ ]:
dataset_for_nltk = []

for filename in os.scandir("datasets/Collection5/"):
    if filename.is_file():
        name, ext = os.path.splitext(filename.path)
        if ext == ".txt":
          with(open(filename.path,"r",encoding='utf-8') as f):
            text = f.read()
            tokens = text.split()

          tagged_tokens = {}

          with(open(name + ".ann","r",encoding='utf-8') as f):
            for line in f:
              parts = line.split()
              tag = parts[1]
              name = ' '.join(parts[4:])
              tagged_tokens[name] = tag

          dataset_for_nltk.append((text, tagged_tokens))


In [ ]:
for entry in dataset_for_nltk:
  pred = [(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(entry[0]))) if hasattr(chunk, 'label')]
  print(entry[1])
  print(pred)
  break

{'Лукашенко': 'PER', '"Белорусской калийной компанией"': 'ORG', 'Александр Лукашенко': 'PER', 'Администрации': 'ORG', 'ЗАО': 'ORG', 'Белоруссии': 'GEOPOLIT', 'Валерия Иванова': 'PER', 'Администрации президента': 'ORG', 'Елены Кудрявец': 'PER', '"Белорусская калийная компания"': 'ORG', 'БКК': 'ORG', 'Иванов': 'PER', 'Кудрявец': 'PER', 'Белорусской калийной компании': 'ORG', 'Российской Федерации': 'GEOPOLIT', '"Беларуськалия"': 'ORG', '"Уралкалия"': 'ORG', 'ОАО "Уралкалий"': 'ORG', 'ОАО "Беларуськалий"': 'ORG', 'ЗАО "Белорусская калийная компания"': 'ORG', 'ОАО "Банк развития Республики Беларусь"': 'ORG', 'Сергей Румас': 'PER', 'КГБ': 'ORG', 'Валерия Вакульчика': 'PER', 'Виктора Лукашенко': 'PER', '"БКК"': 'ORG', 'Сулейманом Керимовым': 'PER', 'Керимов': 'PER', 'Керимовым': 'PER', 'Минске': 'LOC', 'России': 'GEOPOLIT', '"Уралкалию"': 'ORG', '"Беларуськалию"': 'ORG', 'ИА REGNUM': 'MEDIA', '"Союзкалий"': 'ORG', 'Швейцарии': 'GEOPOLIT', '"Международную калийную компанию"': 'ORG', 'Soyuzkal

In [ ]:
!python -m spacy download ru_core_news_lg

2023-08-04 13:10:41.737417: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 68.5 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=6c837a068e2cb0a968e5156756d05549328cda9137d6e6b2a75719196407393d
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [ ]:
nlp = spacy.load('ru_core_news_lg')
doc = nlp(dataset_for_nltk[0][0])
displacy.render(doc, jupyter=True, style='ent')

In [ ]:
!python -m deeppavlov install squad_bert

!python -m deeppavlov install ner_ontonotes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 71.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which

In [ ]:
deeppavlov_ner = build_model(configs.ner, download=True)
rus_document = dataset_for_nltk[0][0]
deeppavlov_ner(rus_document)

AttributeError: ignored

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!locale

LANG=en_US.UTF-8
LANGUAGE=
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC="en_US.UTF-8"
LC_TIME="en_US.UTF-8"
LC_COLLATE="en_US.UTF-8"
LC_MONETARY="en_US.UTF-8"
LC_MESSAGES="en_US.UTF-8"
LC_PAPER="en_US.UTF-8"
LC_NAME="en_US.UTF-8"
LC_ADDRESS="en_US.UTF-8"
LC_TELEPHONE="en_US.UTF-8"
LC_MEASUREMENT="en_US.UTF-8"
LC_IDENTIFICATION="en_US.UTF-8"
LC_ALL=


In [ ]:
import numpy as np

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix


import ru_core_news_lg

import corus
from corus import load_ne5
from razdel import tokenize

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model


In [ ]:
def load_text_patched(path):
    # do not convert \r\n to \n
    with open(path, newline='', encoding='utf-8') as file:
        return file.read()

dir = 'datasets/Collection5/'
# load_ne5 do not accept encoding, but Colab sometimes fails to default to utf-8 for unknown reasons
corus.ne5.load_text = load_text_patched
records = load_ne5(dir)

In [ ]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [ ]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [ ]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 128

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    # ngrams=(1, 5),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [ ]:
embedding_dim = 128

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)

        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)

        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [ ]:
model = modelNER()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['acc'])

In [ ]:
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 99s 7ms/step - loss: 0.2853 - acc: 0.9168 - val_loss: 0.2058 - val_acc: 0.9391
Epoch 2/5
12444/12444 [==============================] - 95s 8ms/step - loss: 0.1209 - acc: 0.9633 - val_loss: 0.2245 - val_acc: 0.9412
Epoch 3/5
12444/12444 [==============================] - 86s 7ms/step - loss: 0.1069 - acc: 0.9659 - val_loss: 0.2563 - val_acc: 0.8848
Epoch 4/5
12444/12444 [==============================] - 96s 8ms/step - loss: 0.1023 - acc: 0.9668 - val_loss: 0.2514 - val_acc: 0.8856
Epoch 5/5
12444/12444 [==============================] - 86s 7ms/step - loss: 0.0991 - acc: 0.9673 - val_loss: 0.2822 - val_acc: 0.8855


In [ ]:
y_pred1 = model.predict(valid_x)
y_pred = np.argmax(y_pred1, axis=1)

print(f'Precision score {precision_score(valid_y, y_pred , average="macro")}')
print(f'Recall score {recall_score(valid_y, y_pred , average="macro")}')
print(f'F1 score {f1_score(valid_y, y_pred , average="macro")}')

2074/2074 [==============================] - 6s 3ms/step
Precision score 0.811798884450888
Recall score 0.7916335073535307
F1 score 0.7852530264378447
